In [1]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [2]:
import torch
import pandas as pd
import numpy as np

In [3]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.8 MB/s eta 0:00:00


In [4]:
df = pd.read_csv('/content/Complaint data annotation (explain)_updated - cd (1).csv')

In [5]:
df.keys()

Index(['id', 'tweet', 'label', 'domain', 'sentiment', 'emotion', 'Severity',
       'Explain'],
      dtype='object')

In [6]:
df.head()

,id,tweet,label,domain,sentiment,emotion,Severity,Explain
0,1,@FC_HELP can I return online purchases to a Ho...,0,apparel,Neutral,other,0,can I return online purchases to a House of Fr...
1,2,@FC_Help Hi - I'm writing a piece for MSN Him ...,0,apparel,Positive,other,0,Hi - I'm writing a piece for MSN Him and wonde...
2,3,@FC_Help i need to check my order,0,apparel,Neutral,other,0,i need to check my order
3,4,@FC_Help I need to get in contact with someone...,1,apparel,Neutral,other,1,I need to get in contact with someone regardin...
4,5,@FC_Help How can I get a hold of you so we can...,0,apparel,Negative,other,0,How can I get a hold of you so we can discuss ...


In [7]:
domain_dict = {
    "other" : 0,
    "services" : 1,
    "random_reply" : 2,
    "software" : 3,
    "retail" : 4,
    "random_tweet" : 5,
    "transport" : 6,
    "cars" : 7,
    "food" : 8,
    "apparel" : 9,
    "electronics" : 10
}

# note to take string lower
senti_dict = {
    'negative' : 0,
    'positive' : 1,
    'neutral' : 2
}

emo_dict = {
    'sadness' : 0, 
    'joy' : 1, 
    'other' : 2, 
    'anger' : 3, 
    'disgust' : 4, 
    'surprise' : 5, 
    'fear' : 6
}

In [8]:
label = []
domain = []
emotion = []
sentiment = []
severity = []
explain = []

for i in range(len(df)):
  if (pd.isna(df['label'][i]) or pd.isna(df['domain'][i]) or pd.isna(df['emotion'][i]) or pd.isna(df['sentiment'][i]) or pd.isna(df['Severity'][i]) or pd.isna(df['Explain'][i])):
    continue
  label.append(df['label'][i])
  domain.append(domain_dict[df['domain'][i]])
  emotion.append(emo_dict[df['emotion'][i]])
  sentiment.append(senti_dict[(df['sentiment'][i]).lower()])
  severity.append(df['Severity'][i])
  explain.append(df['Explain'][i])

In [9]:
from sklearn.model_selection import train_test_split

label_train, label_test, domain_train, domain_test, emotion_train, emotion_test, sentiment_train, sentiment_test, severity_train, severity_test, explain_train, explain_test = train_test_split(label, domain, emotion, sentiment, severity, explain, test_size = 0.2, random_state = 42, shuffle = True)
label_train, label_val, domain_train, domain_val, emotion_train, emotion_val, sentiment_train, sentiment_val, severity_train, severity_val, explain_train, explain_val = train_test_split(label_train, domain_train, emotion_train, sentiment_train, severity_train, explain_train, test_size = 0.2, random_state = 42, shuffle = True)

In [10]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_data(text):
  g_input_ids = []
  g_attention_mask = []

  for line in text:
    inputs = tokenizer(line, return_tensors="pt")

    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    size = input_ids.shape[1]

    if size < 60:
      input_ids = torch.cat((input_ids, torch.zeros(1, 60-size)), dim = 1)
      attention_mask = torch.cat((attention_mask, torch.zeros(1, 60-size)), dim = 1)
    else:
      input_ids = input_ids[:, :60]
      attention_mask = attention_mask[:, :60]
    
    # Why we use np.array()
    g_input_ids.append(np.array(input_ids.reshape(-1)))
    g_attention_mask.append(np.array(attention_mask.reshape(-1)))

  g_input_ids = torch.tensor(g_input_ids)
  g_attention_mask = torch.tensor(g_attention_mask)

  return g_input_ids, g_attention_mask

In [11]:
input_ids_train, attention_mask_train = preprocess_data(explain_train)
input_ids_val, attention_mask_val = preprocess_data(explain_val)
input_ids_test, attention_mask_test = preprocess_data(explain_test)

<ipython-input-10-eb1c066b164e>:28: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  g_input_ids = torch.tensor(g_input_ids)


In [12]:
class Complaint_dataset():
  def __init__(self, input_ids, attention_mask, label, domain, emotion, sentiment, severity):
    self.input_ids = input_ids
    self.attention_mask = attention_mask
    self.label = label
    self.domain = domain
    self.emotion = emotion
    self.sentiment = sentiment
    self.severity = severity
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    
    sample = {
        "input_ids" : torch.tensor(self.input_ids[idx]).long().to(device),
        "attention_mask" : torch.tensor(self.attention_mask[idx]).long().to(device),
        "label" : torch.tensor(self.label[idx]).long().to(device),
        "domain" : torch.tensor(self.domain[idx]).long().to(device),
        "emotion" : torch.tensor(self.emotion[idx]).long().to(device),
        "sentiment" : torch.tensor(self.sentiment[idx]).long().to(device),
        "severity" : torch.tensor(self.severity[idx]).long().to(device)
    }

    return sample

In [13]:
from torch.utils.data import DataLoader

complaint_train = Complaint_dataset(input_ids_train, attention_mask_train, label_train, domain_train, emotion_train, sentiment_train, severity_train)
train_dataloader = DataLoader(complaint_train, batch_size = 32, shuffle = False)

complaint_val = Complaint_dataset(input_ids_val, attention_mask_val, label_val, domain_val, emotion_val, sentiment_val, severity_val)
val_dataloader = DataLoader(complaint_val, batch_size = 32, shuffle = False)

complaint_test = Complaint_dataset(input_ids_test, attention_mask_test, label_test, domain_test, emotion_test, sentiment_test, severity_test)
test_dataloader = DataLoader(complaint_test, batch_size = 32, shuffle = False)

In [14]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

# What is super(Complaint) doing and why the name __init__() as such
# Practise OOPS

class Complaint(nn.Module):
  def __init__(self):
    super(Complaint, self).__init__()
    self.bert_model = BertModel.from_pretrained('bert-base-uncased')
    self.complaint = nn.Linear(768, 2)
    self.severity = nn.Linear(768, 5)
  def forward(self, input_ids, attention_mask):
    data = self.bert_model(input_ids = input_ids, attention_mask = attention_mask).pooler_output
    Complaint = self.complaint(data)
    Severity = self.severity(data)
    return Complaint, Severity

In [15]:
complaint_model = Complaint()
complaint_model = complaint_model.to(device)
complaint_model.train()

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(complaint_model.parameters(), lr = 1e-5, weight_decay = 0)
epochs = 20
min_val_loss = 10

complaint_model.train()

for i in range(epochs):

  for data in train_dataloader:
    complaint_model.zero_grad()
    comp_train = data["label"].to(device)
    sev_train = data["severity"].to(device)
    emo_train = data["emotion"].to(device)
    senti_train = data["sentiment"].to(device)
    comp_out, sev_out = complaint_model.forward(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device))

    loss_train = loss_func(comp_out, comp_train) + loss_func(sev_out, sev_train)
    loss_train.backward()
    optimizer.step()

    complaint_model.eval()

    with torch.no_grad():
      total_loss_val = 0
      for data in val_dataloader:
        comp_val = data["label"].to(device)
        sev_val = data["severity"].to(device)
        emo_val = data["emotion"].to(device)
        senti_val = data["sentiment"].to(device)
        comp_val_out, sev_val_out = complaint_model(input_ids = data["input_ids"].to(device), attention_mask = data["attention_mask"].to(device))

        val_loss = loss_func(comp_val_out, comp_val) + loss_func(sev_val_out, sev_val)

        total_val = comp_val.size(0)
        total_loss_val += val_loss.item()

      val_loss = total_loss_val / total_val

    if ((min_val_loss-val_loss) > 1e-4):
      min_val_loss = val_loss
      torch.save(complaint_model.state_dict(), "bert_model.pt")

  print(f"Epoch : {i+1}")
  print(f"Validation loss : {val_loss}")

  complaint_model.load_state_dict(torch.load("bert_model.pt"))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-12-aa0febf124e5>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.cl

Epoch : 1
Validation loss : 3.2234461307525635
Epoch : 2
Validation loss : 2.836848482489586
Epoch : 3
Validation loss : 2.588334619998932
Epoch : 4
Validation loss : 2.6125908121466637
Epoch : 5
Validation loss : 2.75778342038393
Epoch : 6
Validation loss : 3.2326973900198936
Epoch : 7
Validation loss : 2.814572587609291
Epoch : 8
Validation loss : 2.932586297392845
Epoch : 9
Validation loss : 2.8130855336785316
Epoch : 10
Validation loss : 3.0867473110556602
Epoch : 11
Validation loss : 2.762418657541275
Epoch : 12
Validation loss : 2.9909066781401634
Epoch : 13
Validation loss : 2.775322027504444
Epoch : 14
Validation loss : 3.188516542315483
Epoch : 15
Validation loss : 2.81193508207798
Epoch : 16
Validation loss : 2.9587571173906326
Epoch : 17
Validation loss : 2.8014901280403137
Epoch : 18
Validation loss : 3.343397356569767
Epoch : 19
Validation loss : 2.846118301153183
Epoch : 20
Validation loss : 2.9938517808914185


In [16]:
from sklearn.metrics import *

In [17]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 10.0 MB/s eta 0:00:00


In [18]:
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryF1Score, BinaryRecall
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassF1Score, MulticlassRecall

comp_accuracy = BinaryAccuracy()
comp_precision = BinaryPrecision()
comp_f1 = BinaryF1Score()
comp_recall = BinaryRecall() 

sev_accuracy = MulticlassAccuracy(num_classes=5)
sev_precision = MulticlassPrecision(num_classes=5)
sev_f1 = MulticlassF1Score(num_classes=5)
sev_recall = MulticlassRecall(num_classes=5)

with torch.no_grad():
    for data in test_dataloader:
      lab_comp = data["label"]
      lab_sev = data["severity"]

      comp_out, sev_out = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])

      _, pred_comp = torch.max(comp_out.data,1)
      _, pred_sev = torch.max(sev_out.data,1)

      comp_accuracy.update(pred_comp.cpu(), lab_comp.cpu())
      comp_precision.update(pred_comp.cpu(), lab_comp.cpu()) 
      comp_f1.update(pred_comp.cpu(), lab_comp.cpu())
      comp_recall.update(pred_comp.cpu(), lab_comp.cpu())

      sev_accuracy.update(pred_sev.cpu(), lab_sev.cpu())
      sev_precision.update(pred_sev.cpu(), lab_sev.cpu())
      sev_f1.update(pred_sev.cpu(), lab_sev.cpu())
      sev_recall.update(pred_sev.cpu(), lab_sev.cpu())

    comp_test_accuracy = comp_accuracy.compute()
    comp_test_precision = comp_precision.compute()
    comp_test_f1 = comp_f1.compute()
    comp_test_recall = comp_recall.compute() 

    sev_test_accuracy = sev_accuracy.compute()
    sev_test_precision = sev_precision.compute()
    sev_test_f1 = sev_f1.compute()
    sev_test_recall = sev_recall.compute()

    print(f"comp_test_accuracy : {comp_test_accuracy}")
    print(f"comp_test_precision : {comp_test_precision}")
    print(f"comp_test_f1 : {comp_test_f1}")
    print(f"comp_test_recall : {comp_test_recall}") 

    print(f"sev_test_accuracy : {sev_test_accuracy}")
    print(f"sev_test_precision : {sev_test_precision}")
    print(f"sev_test_f1 : {sev_test_f1}")
    print(f"sev_test_recall : {sev_test_recall}") 

<ipython-input-12-aa0febf124e5>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids" : torch.tensor(self.input_ids[idx]).long().to(device),
<ipython-input-12-aa0febf124e5>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask" : torch.tensor(self.attention_mask[idx]).long().to(device),


comp_test_accuracy : 0.8681159615516663
comp_test_precision : 0.7941176295280457
comp_test_f1 : 0.8260038495063782
comp_test_recall : 0.8605577945709229
sev_test_accuracy : 0.3460594713687897
sev_test_precision : 0.2982306480407715
sev_test_f1 : 0.31934285163879395
sev_test_recall : 0.3460594713687897


In [19]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import DataLoader

In [20]:
embed_input = "bert"
bitmask_input = "comp+sent"
hid_size = 150

In [21]:
bitmasks = {
    "comp" : 1,
    "sent" : 2,
    "emo" : 4,
    "comp+sent" : 3,
    "comp+emo" : 5,
    "comp+sev" : 9,
    "comp+sent+emo" : 7,
    "comp+sent+sev" : 11,
    "comp+emo+sev" : 13,
    "comp+emo+sev+sent" : 15
}

embeds = {
    "bert" : "bert-base-uncased",
    "roberta" : "roberta-base"
}

In [22]:
learning_rate = 1e-5
epochs = 20
min_val_loss = 10

In [23]:
class complaint(nn.Module):
  def __init__(self, hidden_size, bitmask, embedding):
    super(complaint, self).__init__()
    self.emb_model = AutoModel.from_pretrained(embedding)
    self.dropout = nn.Dropout(p = 0.5)
    self.hidden = nn.Linear(768, hidden_size)
    self.comp = nn.Linear(hidden_size, 2) #Complaint Output_Size = 2
    self.sent = nn.Linear(hidden_size, 3) #Sentiment Output_Size = 3
    self.sev = nn.Linear(hidden_size, 5) #Severity Level Output_Size = 5
    self.emo = nn.Linear(hidden_size, 7) #Emotional Level Output_Size = 7
    self.bitmask = bitmask

  def forward_comp(self, x):
    out = self.comp(x)
    return out
  
  def forward_sent(self, x):
    out = self.sent(x)
    return out
  
  def forward_sev(self, x):
    out = self.sev(x)
    return out
  
  def forward_emo(self, x):
    out = self.emo(x)
    return out

  def forward(self, input_ids, attention_mask, bitmask):
    data = self.dropout(self.emb_model(input_ids=input_ids, attention_mask=attention_mask).pooler_output)
    out = self.hidden(data)
    if (bitmask == bitmasks["comp"]):
      out = self.forward_comp(out)
      return out
    
    if (bitmask == bitmasks["sent"]):
      out = self.forward_sent(out)
      return out
    
    if (bitmask == bitmasks["emo"]):
      out = self.forward_emo(out)
      return out

    elif (bitmask == bitmasks["comp+sent"]):
      out1 = self.forward_comp(out)
      out2 = self.forward_sent(out)
      return out1, out2
    elif(bitmask == bitmasks["comp+emo"]):
      out1 = self.forward_comp(out)
      out2 = self.forward_emo(out)
      return out1, out2
    elif(bitmask == bitmasks["comp+sev"]):
      out1 = self.forward_comp(out)
      out2 = self.forward_sev(out)
      return out1, out2
    elif(bitmask == bitmasks["comp+sent+emo"]):
      out1 = self.forward_comp(out)
      out2 = self.forward_sent(out)
      out3 = self.forward_emo(out)
      return out1, out2, out3
    elif(bitmask == bitmasks["comp+sent+sev"]):
      out1 = self.forward_comp(out)
      out2 = self.forward_sent(out)
      out3 = self.forward_sev(out)
      return out1, out2, out3
    elif(bitmask == bitmasks["comp+emo+sev"]):
      out1 = self.forward_comp(out)
      out2 = self.forward_emo(out)
      out3 = self.forward_sev(out)
      return out1, out2, out3
    elif(bitmask == bitmasks["comp+emo+sev+sent"]):
      out1 = self.forward_comp(out)
      out2 = self.forward_emo(out)
      out3 = self.forward_sev(out)
      out4 = self.forward_sent(out)
      return out1, out2, out3, out4

In [24]:
df = pd.read_excel("/content/aspect_complain_formatted.xlsx", "new-FINCORP")

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [25]:
doc = []
label = []

label_number = {
    "comp" : 0,
    "sent" : 1,
    "sev" : 2,
    "emo" : 3
}

sent_num = {
    "positive" : 2,
    "neutral" : 1,
    "negative" : 0
}

sev_num = {
    "accusation" : 0,
    "blame" : 1,
    "disapproval" : 2,
    "no explicit reproach" : 3,
    "non-complaint" : 4
}

emo_num = {
    "anger" : 0,
    "disgust" : 1,
    "fear" : 2,
    "happiness" : 3,
    "other" : 4,
    "sadness" : 5,
    "surprise" : 6
}

for i in range(len(df)):
  doc.append((df["Complaint/ Opinion"])[i])
  temp = []
  temp.append((df["Over-all_Complaint Label"])[i])
  temp.append(sent_num[((df["Sentiment"])[i]).lower()])
  temp.append(sev_num[((df["Severity level"])[i]).lower()])
  temp.append(emo_num[((df["Emotion"])[i]).lower()])
  label.append(temp)

doc_tr, doc_test, label_tr, label_test = train_test_split(doc, label, test_size=0.2, random_state = 42, shuffle=True)
doc_train, doc_val, label_train, label_val = train_test_split(doc_tr,label_tr, test_size=0.125, random_state = 42, shuffle=True)

In [26]:
tokenizer = AutoTokenizer.from_pretrained(embeds[embed_input])

In [27]:
def preprocess_data(doc):
  
  g_input_ids = []
  g_attention_mask = []
  
  for x in doc:
    inputs = tokenizer(x, return_tensors="pt")

    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    size = input_ids.shape[1]

    if size < 60:
      input_ids = torch.cat((input_ids, torch.zeros(1, 60 - size)), dim=1)
      attention_mask = torch.cat((attention_mask, torch.zeros(1, 60 - size)), dim=1)
    else:
      input_ids = input_ids[:, :60]
      attention_mask = attention_mask[:, :60]
    
    g_input_ids.append(np.array(input_ids.reshape(-1)))
    g_attention_mask.append(np.array(attention_mask.reshape(-1)))
  
  g_input_ids = torch.tensor(g_input_ids)
  g_attention_mask = torch.tensor(g_attention_mask)

  return g_input_ids, g_attention_mask

In [28]:
input_ids_train,attention_mask_train = preprocess_data(doc_train)
input_ids_val, attention_mask_val = preprocess_data(doc_val)
input_ids_test, attention_mask_test = preprocess_data(doc_test)

In [29]:
class complaint_dataset(torch.utils.data.Dataset):
  def __init__(self, g_input_ids, g_attention_mask, label_comp, label_sent, label_sev, label_emo):
    self.input_ids = g_input_ids
    self.attention_mask = g_attention_mask
    self.label_comp = label_comp
    self.label_sent = label_sent
    self.label_sev = label_sev
    self.label_emo = label_emo
  
  def __len__(self):
    return len(self.label_comp)
  
  def __getitem__(self, idx):

    if torch.is_tensor(idx):
      idx = idx.tolist()
    
    sample = {
        # "input_ids" : torch.tensor(self.input_ids[idx]).long().to(device),
        # "attention_mask" : torch.tensor(self.attention_mask[idx]).long().to(device),
        # "label_comp" : torch.tensor(self.label_comp[idx]).long().to(device),
        # "label_sent" : torch.tensor(self.label_sent[idx]).long().to(device),
        # "label_sev" : torch.tensor(self.label_sev[idx]).long().to(device),
        # "label_emo" : torch.tensor(self.label_emo[idx]).long().to(device)

        "input_ids" : torch.tensor(self.input_ids[idx]).long(),
        "attention_mask" : torch.tensor(self.attention_mask[idx]).long(),
        "label_comp" : torch.tensor(self.label_comp[idx]).long(),
        "label_sent" : torch.tensor(self.label_sent[idx]).long(),
        "label_sev" : torch.tensor(self.label_sev[idx]).long(),
        "label_emo" : torch.tensor(self.label_emo[idx]).long()
    }

    return sample


In [30]:
train_comp = []
train_sent = []
train_sev = []
train_emo = []
val_comp = []
val_sent = []
val_sev = []
val_emo = []
test_comp = []
test_sent = []
test_sev = []
test_emo = []

for x in label_train:
  train_comp.append(x[0])
  train_sent.append(x[1])
  train_sev.append(x[2])
  train_emo.append(x[3])

for x in label_val:
  val_comp.append(x[0])
  val_sent.append(x[1])
  val_sev.append(x[2])
  val_emo.append(x[3])

for x in label_test:
  test_comp.append(x[0])
  test_sent.append(x[1])
  test_sev.append(x[2])
  test_emo.append(x[3])

In [31]:
complaint_train = complaint_dataset(input_ids_train, attention_mask_train, train_comp, train_sent, train_sev, train_emo)
train_dataloader = DataLoader(complaint_train, batch_size = 32, shuffle = False)

complaint_val = complaint_dataset(input_ids_val, attention_mask_val, val_comp, val_sent, val_sev, val_emo)
val_dataloader = DataLoader(complaint_val, batch_size = 32 ,shuffle = False)

complaint_test = complaint_dataset(input_ids_test, attention_mask_test, test_comp, test_sent, test_sev, test_emo)
test_dataloader = DataLoader(complaint_test, batch_size = 32, shuffle = False)

In [32]:
class CELoss(nn.Module):

    def __init__(self):
        super().__init__()
        self.xent_loss = nn.CrossEntropyLoss()

    def forward(self, outputs, targets):
        return self.xent_loss(outputs, targets)


class SupConLoss(nn.Module):

    def __init__(self, alpha, temp):
        super().__init__()
        self.xent_loss = nn.CrossEntropyLoss()
        self.alpha = alpha
        self.temp = temp

    def nt_xent_loss(self, anchor, target, labels):
        with torch.no_grad():
            labels = labels.unsqueeze(-1)
            mask = torch.eq(labels, labels.transpose(0, 1))
            # delete diag elem
            mask = mask ^ torch.diag_embed(torch.diag(mask))
        # compute logits
        anchor_dot_target = torch.einsum('bd,cd->bc', anchor, target) / self.temp
        # delete diag elem
        anchor_dot_target = anchor_dot_target - torch.diag_embed(torch.diag(anchor_dot_target))
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_target, dim=1, keepdim=True)
        logits = anchor_dot_target - logits_max.detach()
        # compute log prob
        exp_logits = torch.exp(logits)
        # mask out positives
        logits = logits * mask
        log_prob = logits - torch.log(exp_logits.sum(dim=1, keepdim=True) + 1e-12)
        # in case that mask.sum(1) is zero
        mask_sum = mask.sum(dim=1)
        mask_sum = torch.where(mask_sum == 0, torch.ones_like(mask_sum), mask_sum)
        # compute log-likelihood
        pos_logits = (mask * log_prob).sum(dim=1) / mask_sum.detach()
        loss = -1 * pos_logits.mean()
        return loss

    def forward(self, outputs, targets):
        ce_loss = (1 - self.alpha) * self.xent_loss(outputs, targets)
        cl_loss = self.alpha * self.nt_xent_loss(outputs, outputs, targets)
        return ce_loss + cl_loss


class DualLoss(SupConLoss):

    def __init__(self, alpha, temp):
        super().__init__(alpha, temp)

    def forward(self, outputs, targets):
        normed_cls_feats = F.normalize(outputs['cls_feats'], dim=-1)
        normed_label_feats = F.normalize(outputs['label_feats'], dim=-1)
        normed_pos_label_feats = torch.gather(normed_label_feats, dim=1, index=targets.unsqueeze(-1).expand(-1, 1, normed_label_feats.size(-1))).squeeze(1)
        ce_loss = (1 - self.alpha) * self.xent_loss(outputs['predicts'], targets)
        cl_loss_1 = 0.5 * self.alpha * self.nt_xent_loss(normed_pos_label_feats, normed_cls_feats, targets)
        cl_loss_2 = 0.5 * self.alpha * self.nt_xent_loss(normed_cls_feats, normed_pos_label_feats, targets)
        return ce_loss + cl_loss_1 + cl_loss_2

In [33]:
criterion = SupConLoss(0.1, 0.1)

In [34]:
from sklearn.metrics import *

In [35]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
complaint_model = complaint(hidden_size = hid_size, bitmask=bitmasks[bitmask_input], embedding=embeds[embed_input])


model_name = embed_input + "_" + bitmask_input + ".pt"

for i in range(epochs):

  total_loss_train = 0
  total_train = 0

 
      

  complaint_model.load_state_dict(torch.load(model_name))

In [38]:
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryF1Score, BinaryRecall
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassF1Score, MulticlassRecall

comp_accuracy = BinaryAccuracy()
comp_precision = BinaryPrecision()
comp_f1 = BinaryF1Score()
comp_recall = BinaryRecall()

sent_accuracy = MulticlassAccuracy(num_classes=3)
sent_precision = MulticlassPrecision(num_classes=3)
sent_f1 = MulticlassF1Score(num_classes=3)
sent_recall = MulticlassRecall(num_classes=3)

sev_accuracy = MulticlassAccuracy(num_classes=5)
sev_precision = MulticlassPrecision(num_classes=5)
sev_f1 = MulticlassF1Score(num_classes=5)
sev_recall = MulticlassRecall(num_classes=5)

emo_accuracy = MulticlassAccuracy(num_classes=7)
emo_precision = MulticlassPrecision(num_classes=7)
emo_f1 = MulticlassF1Score(num_classes=7)
emo_recall = MulticlassRecall(num_classes=7)

with torch.no_grad():
  if(bitmask_input == "comp"):
    for data in test_dataloader:
      lab_comp = data["label_comp"]
      lab_sent = data["label_sent"]
      lab_sev = data["label_sev"]
      lab_emo = data["label_emo"]
      output = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])
      loss = criterion(output, lab_comp)

      _, pred_comp = torch.max(output.data,1)

      comp_accuracy.update(pred_comp.cpu(), lab_comp.cpu())
      comp_precision.update(pred_comp.cpu(), lab_comp.cpu())
      comp_f1.update(pred_comp.cpu(), lab_comp.cpu())
      comp_recall.update(pred_comp.cpu(), lab_comp.cpu())

    comp_test_accuracy = comp_accuracy.compute()
    comp_test_precision = comp_precision.compute()
    comp_test_f1 = comp_f1.compute()
    comp_test_recall = comp_recall.compute()
    print(f"comp_test_accuracy : {comp_test_accuracy}")
    print(f"comp_test_precision : {comp_test_precision}")
    print(f"comp_test_f1 : {comp_test_f1}")
    print(f"comp_test_recall : {comp_test_recall}")
  
  elif(bitmask_input == "sent"):
    for data in test_dataloader:
      lab_comp = data["label_comp"]
      lab_sent = data["label_sent"]
      lab_sev = data["label_sev"]
      lab_emo = data["label_emo"]
      out = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])
      loss = criterion(out, lab_sent)

      _, pred_sent = torch.max(out2.data,1)

      sent_accuracy.update(pred_sent.cpu(), lab_sent.cpu())
      sent_precision.update(pred_sent.cpu(), lab_sent.cpu())
      sent_f1.update(pred_sent.cpu(), lab_sent.cpu())
      sent_recall.update(pred_sent.cpu(), lab_sent.cpu())

    sent_test_accuracy = sent_accuracy.compute()
    sent_test_precision = sent_precision.compute()
    sent_test_f1 = sent_f1.compute()
    sent_test_recall = sent_recall.compute()

    print(f"sent_test_accuracy : {sent_test_accuracy}")
    print(f"sent_test_precision : {sent_test_precision}")
    print(f"sent_test_f1 : {sent_test_f1}")
    print(f"sent_test_recall : {sent_test_recall}")

  elif(bitmask_input == "emo"):
    for data in test_dataloader:
      lab_comp = data["label_comp"]
      lab_sent = data["label_sent"]
      lab_sev = data["label_sev"]
      lab_emo = data["label_emo"]
      out = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])
      loss = criterion(out, lab_emo)

      _, pred_emo = torch.max(out3.data,1)

      emo_accuracy.update(pred_emo.cpu(), lab_emo.cpu())
      emo_precision.update(pred_emo.cpu(), lab_emo.cpu())
      emo_f1.update(pred_emo.cpu(), lab_emo.cpu())
      emo_recall.update(pred_emo.cpu(), lab_emo.cpu())
  
    emo_test_accuracy = emo_accuracy.compute()
    emo_test_precision = emo_precision.compute()
    emo_test_f1 = emo_f1.compute()
    emo_test_recall = emo_recall.compute()
    
    print(f"emo_test_accuracy : {emo_test_accuracy}")
    print(f"emo_test_precision : {emo_test_precision}")
    print(f"emo_test_f1 : {emo_test_f1}")
    print(f"emo_test_recall : {emo_test_recall}")

  elif(bitmask_input == "comp+sent"):
    for data in test_dataloader:
      lab_comp = data["label_comp"]
      lab_sent = data["label_sent"]
      lab_sev = data["label_sev"]
      lab_emo = data["label_emo"]
      out1, out2 = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])
      loss = criterion(out1, lab_comp) + criterion(out2, lab_sent)

      _, pred_comp = torch.max(out1.data,1)
      _, pred_sent = torch.max(out2.data,1)

      comp_accuracy.update(pred_comp.cpu(), lab_comp.cpu())
      comp_precision.update(pred_comp.cpu(), lab_comp.cpu())
      comp_f1.update(pred_comp.cpu(), lab_comp.cpu())
      comp_recall.update(pred_comp.cpu(), lab_comp.cpu())

      sent_accuracy.update(pred_sent.cpu(), lab_sent.cpu())
      sent_precision.update(pred_sent.cpu(), lab_sent.cpu())
      sent_f1.update(pred_sent.cpu(), lab_sent.cpu())
      sent_recall.update(pred_sent.cpu(), lab_sent.cpu())
      comp_test_accuracy = comp_accuracy.compute()

    comp_test_accuracy = comp_accuracy.compute()
    comp_test_precision = comp_precision.compute()
    comp_test_f1 = comp_f1.compute()
    comp_test_recall = comp_recall.compute()

    sent_test_accuracy = sent_accuracy.compute()
    sent_test_precision = sent_precision.compute()
    sent_test_f1 = sent_f1.compute()
    sent_test_recall = sent_recall.compute()

    print(f"comp_test_accuracy : {comp_test_accuracy}")
    print(f"comp_test_precision : {comp_test_precision}")
    print(f"comp_test_f1 : {comp_test_f1}")
    print(f"comp_test_recall : {comp_test_recall}")

    print(f"sent_test_accuracy : {sent_test_accuracy}")
    print(f"sent_test_precision : {sent_test_precision}")
    print(f"sent_test_f1 : {sent_test_f1}")
    print(f"sent_test_recall : {sent_test_recall}")


  elif(bitmask_input == "comp+emo"):
    for data in test_dataloader:
      lab_comp = data["label_comp"]
      lab_sent = data["label_sent"]
      lab_sev = data["label_sev"]
      lab_emo = data["label_emo"]
      out1, out2 = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])
      loss = criterion(out1, lab_comp) + criterion(out2, lab_emo)

      _, pred_comp = torch.max(out1.data,1)
      _, pred_emo = torch.max(out2.data,1)

      comp_accuracy.update(pred_comp.cpu(), lab_comp.cpu())
      comp_precision.update(pred_comp.cpu(), lab_comp.cpu())
      comp_f1.update(pred_comp.cpu(), lab_comp.cpu())
      comp_recall.update(pred_comp.cpu(), lab_comp.cpu())

      emo_accuracy.update(pred_emo.cpu(), lab_emo.cpu())
      emo_precision.update(pred_emo.cpu(), lab_emo.cpu())
      emo_f1.update(pred_emo.cpu(), lab_emo.cpu())
      emo_recall.update(pred_emo.cpu(), lab_emo.cpu())

    comp_test_accuracy = comp_accuracy.compute()
    comp_test_precision = comp_precision.compute()
    comp_test_f1 = comp_f1.compute()
    comp_test_recall = comp_recall.compute()

    emo_test_accuracy = emo_accuracy.compute()
    emo_test_precision = emo_precision.compute()
    emo_test_f1 = emo_f1.compute()
    emo_test_recall = emo_recall.compute()

    print(f"comp_test_accuracy : {comp_test_accuracy}")
    print(f"comp_test_precision : {comp_test_precision}")
    print(f"comp_test_f1 : {comp_test_f1}")
    print(f"comp_test_recall : {comp_test_recall}")

    print(f"emo_test_accuracy : {emo_test_accuracy}")
    print(f"emo_test_precision : {emo_test_precision}")
    print(f"emo_test_f1 : {emo_test_f1}")
    print(f"emo_test_recall : {emo_test_recall}")

  elif(bitmask_input == "comp+sev"):
    for data in test_dataloader:
      lab_comp = data["label_comp"]
      lab_sent = data["label_sent"]
      lab_sev = data["label_sev"]
      lab_emo = data["label_emo"]
      out1, out2 = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])
      loss = criterion(out1, lab_comp) + criterion(out2, lab_sev)

      _, pred_comp = torch.max(out1.data,1)
      _, pred_sev = torch.max(out2.data,1)

      comp_accuracy.update(pred_comp.cpu(), lab_comp.cpu())
      comp_precision.update(pred_comp.cpu(), lab_comp.cpu())
      comp_f1.update(pred_comp.cpu(), lab_comp.cpu())
      comp_recall.update(pred_comp.cpu(), lab_comp.cpu())

      sev_accuracy.update(pred_sev.cpu(), lab_sev.cpu())
      sev_precision.update(pred_sev.cpu(), lab_sev.cpu())
      sev_f1.update(pred_sev.cpu(), lab_sev.cpu())
      sev_recall.update(pred_sev.cpu(), lab_sev.cpu())

    comp_test_accuracy = comp_accuracy.compute()
    comp_test_precision = comp_precision.compute()
    comp_test_f1 = comp_f1.compute()
    comp_test_recall = comp_recall.compute()

    sev_test_accuracy = sev_accuracy.compute()
    sev_test_precision = sev_precision.compute()
    sev_test_f1 = sev_f1.compute()
    sev_test_recall = sev_recall.compute()

    print(f"comp_test_accuracy : {comp_test_accuracy}")
    print(f"comp_test_precision : {comp_test_precision}")
    print(f"comp_test_f1 : {comp_test_f1}")
    print(f"comp_test_recall : {comp_test_recall}")

    print(f"sev_test_accuracy : {sev_test_accuracy}")
    print(f"sev_test_precision : {sev_test_precision}")
    print(f"sev_test_f1 : {sev_test_f1}")
    print(f"sev_test_recall : {sev_test_recall}")

  elif(bitmask_input == "comp+sent+emo"):
    for data in test_dataloader:
      lab_comp = data["label_comp"]
      lab_sent = data["label_sent"]
      lab_sev = data["label_sev"]
      lab_emo = data["label_emo"]
      out1, out2, out3 = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])
      loss = criterion(out1, lab_comp) + criterion(out2, lab_sent) + criterion(out3, lab_emo)

      _, pred_comp = torch.max(out1.data,1)
      _, pred_sent = torch.max(out2.data,1)
      _, pred_emo = torch.max(out3.data,1)

      comp_accuracy.update(pred_comp.cpu(), lab_comp.cpu())
      comp_precision.update(pred_comp.cpu(), lab_comp.cpu())
      comp_f1.update(pred_comp.cpu(), lab_comp.cpu())
      comp_recall.update(pred_comp.cpu(), lab_comp.cpu())

      sent_accuracy.update(pred_sent.cpu(), lab_sent.cpu())
      sent_precision.update(pred_sent.cpu(), lab_sent.cpu())
      sent_f1.update(pred_sent.cpu(), lab_sent.cpu())
      sent_recall.update(pred_sent.cpu(), lab_sent.cpu())

      emo_accuracy.update(pred_emo.cpu(), lab_emo.cpu())
      emo_precision.update(pred_emo.cpu(), lab_emo.cpu())
      emo_f1.update(pred_emo.cpu(), lab_emo.cpu())
      emo_recall.update(pred_emo.cpu(), lab_emo.cpu())
    
    comp_test_accuracy = comp_accuracy.compute()
    comp_test_precision = comp_precision.compute()
    comp_test_f1 = comp_f1.compute()
    comp_test_recall = comp_recall.compute()

    emo_test_accuracy = emo_accuracy.compute()
    emo_test_precision = emo_precision.compute()
    emo_test_f1 = emo_f1.compute()
    emo_test_recall = emo_recall.compute()

    sent_test_accuracy = sent_accuracy.compute()
    sent_test_precision = sent_precision.compute()
    sent_test_f1 = sent_f1.compute()
    sent_test_recall = sent_recall.compute()

    print(f"comp_test_accuracy : {comp_test_accuracy}")
    print(f"comp_test_precision : {comp_test_precision}")
    print(f"comp_test_f1 : {comp_test_f1}")
    print(f"comp_test_recall : {comp_test_recall}")

    print(f"sent_test_accuracy : {sent_test_accuracy}")
    print(f"sent_test_precision : {sent_test_precision}")
    print(f"sent_test_f1 : {sent_test_f1}")
    print(f"sent_test_recall : {sent_test_recall}")

    print(f"emo_test_accuracy : {emo_test_accuracy}")
    print(f"emo_test_precision : {emo_test_precision}")
    print(f"emo_test_f1 : {emo_test_f1}")
    print(f"emo_test_recall : {emo_test_recall}")

  elif(bitmask_input == "comp+sent+sev"):
    for data in test_dataloader:
      lab_comp = data["label_comp"]
      lab_sent = data["label_sent"]
      lab_sev = data["label_sev"]
      lab_emo = data["label_emo"]
      out1, out2, out3 = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])
      loss = criterion(out1, lab_comp) + criterion(out2, lab_sent) + criterion(out3, lab_sev)

      _, pred_comp = torch.max(out1.data,1)
      _, pred_sent = torch.max(out2.data,1)
      _, pred_sev = torch.max(out3.data,1)

      comp_accuracy.update(pred_comp.cpu(), lab_comp.cpu())
      comp_precision.update(pred_comp.cpu(), lab_comp.cpu())
      comp_f1.update(pred_comp.cpu(), lab_comp.cpu())
      comp_recall.update(pred_comp.cpu(), lab_comp.cpu())

      sent_accuracy.update(pred_sent.cpu(), lab_sent.cpu())
      sent_precision.update(pred_sent.cpu(), lab_sent.cpu())
      sent_f1.update(pred_sent.cpu(), lab_sent.cpu())
      sent_recall.update(pred_sent.cpu(), lab_sent.cpu())

      sev_accuracy.update(pred_sev.cpu(), lab_sev.cpu())
      sev_precision.update(pred_sev.cpu(), lab_sev.cpu())
      sev_f1.update(pred_sev.cpu(), lab_sev.cpu())
      sev_recall.update(pred_sev.cpu(), lab_sev.cpu())

    comp_test_accuracy = comp_accuracy.compute()
    comp_test_precision = comp_precision.compute()
    comp_test_f1 = comp_f1.compute()
    comp_test_recall = comp_recall.compute()

    sev_test_accuracy = sev_accuracy.compute()
    sev_test_precision = sev_precision.compute()
    sev_test_f1 = sev_f1.compute()
    sev_test_recall = sev_recall.compute()

    sent_test_accuracy = sent_accuracy.compute()
    sent_test_precision = sent_precision.compute()
    sent_test_f1 = sent_f1.compute()
    sent_test_recall = sent_recall.compute()

    print(f"comp_test_accuracy : {comp_test_accuracy}")
    print(f"comp_test_precision : {comp_test_precision}")
    print(f"comp_test_f1 : {comp_test_f1}")
    print(f"comp_test_recall : {comp_test_recall}")

    print(f"sent_test_accuracy : {sent_test_accuracy}")
    print(f"sent_test_precision : {sent_test_precision}")
    print(f"sent_test_f1 : {sent_test_f1}")
    print(f"sent_test_recall : {sent_test_recall}")

    print(f"sev_test_accuracy : {sev_test_accuracy}")
    print(f"sev_test_precision : {sev_test_precision}")
    print(f"sev_test_f1 : {sev_test_f1}")
    print(f"sev_test_recall : {sev_test_recall}")

  elif(bitmask_input == "comp+emo+sev"):
    for data in test_dataloader:
      lab_comp = data["label_comp"]
      lab_sent = data["label_sent"]
      lab_sev = data["label_sev"]
      lab_emo = data["label_emo"]
      out1, out2, out3 = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])
      loss = criterion(out1, lab_comp) + criterion(out2, lab_emo) + criterion(out3, lab_sev)

      _, pred_comp = torch.max(out1.data,1)
      _, pred_emo = torch.max(out2.data,1)
      _, pred_sev = torch.max(out3.data,1)

      comp_accuracy.update(pred_comp.cpu(), lab_comp.cpu())
      comp_precision.update(pred_comp.cpu(), lab_comp.cpu())
      comp_f1.update(pred_comp.cpu(), lab_comp.cpu())
      comp_recall.update(pred_comp.cpu(), lab_comp.cpu())

      emo_accuracy.update(pred_emo.cpu(), lab_emo.cpu())
      emo_precision.update(pred_emo.cpu(), lab_emo.cpu())
      emo_f1.update(pred_emo.cpu(), lab_emo.cpu())
      emo_recall.update(pred_emo.cpu(), lab_emo.cpu())

      sev_accuracy.update(pred_sev.cpu(), lab_sev.cpu())
      sev_precision.update(pred_sev.cpu(), lab_sev.cpu())
      sev_f1.update(pred_sev.cpu(), lab_sev.cpu())
      sev_recall.update(pred_sev.cpu(), lab_sev.cpu())
    
    comp_test_accuracy = comp_accuracy.compute()
    comp_test_precision = comp_precision.compute()
    comp_test_f1 = comp_f1.compute()
    comp_test_recall = comp_recall.compute()

    sev_test_accuracy = sev_accuracy.compute()
    sev_test_precision = sev_precision.compute()
    sev_test_f1 = sev_f1.compute()
    sev_test_recall = sev_recall.compute()

    emo_test_accuracy = emo_accuracy.compute()
    emo_test_precision = emo_precision.compute()
    emo_test_f1 = emo_f1.compute()
    emo_test_recall = emo_recall.compute()

    print(f"comp_test_accuracy : {comp_test_accuracy}")
    print(f"comp_test_precision : {comp_test_precision}")
    print(f"comp_test_f1 : {comp_test_f1}")
    print(f"comp_test_recall : {comp_test_recall}")

    print(f"emo_test_accuracy : {emo_test_accuracy}")
    print(f"emo_test_precision : {emo_test_precision}")
    print(f"emo_test_f1 : {emo_test_f1}")
    print(f"emo_test_recall : {emo_test_recall}")

    print(f"sev_test_accuracy : {sev_test_accuracy}")
    print(f"sev_test_precision : {sev_test_precision}")
    print(f"sev_test_f1 : {sev_test_f1}")
    print(f"sev_test_recall : {sev_test_recall}")

  elif(bitmask_input == "comp+emo+sev+sent"):
    for data in test_dataloader:
      lab_comp = data["label_comp"]
      lab_sent = data["label_sent"]
      lab_sev = data["label_sev"]
      lab_emo = data["label_emo"]
      out1, out2, out3, out4 = complaint_model.forward(input_ids=data["input_ids"], attention_mask=data["attention_mask"])
      loss = criterion(out1, lab_comp) + criterion(out2, lab_emo) + criterion(out3, lab_sev) + criterion(out4, lab_sent)

      _, pred_comp = torch.max(out1.data,1)
      _, pred_emo = torch.max(out2.data,1)
      _, pred_sev = torch.max(out3.data,1)
      _, pred_sent = torch.max(out4.data,1)

      comp_accuracy.update(pred_comp.cpu(), lab_comp.cpu())
      comp_precision.update(pred_comp.cpu(), lab_comp.cpu())
      comp_f1.update(pred_comp.cpu(), lab_comp.cpu())
      comp_recall.update(pred_comp.cpu(), lab_comp.cpu())

      emo_accuracy.update(pred_emo.cpu(), lab_emo.cpu())
      emo_precision.update(pred_emo.cpu(), lab_emo.cpu())
      emo_f1.update(pred_emo.cpu(), lab_emo.cpu())
      emo_recall.update(pred_emo.cpu(), lab_emo.cpu())

      sev_accuracy.update(pred_sev.cpu(), lab_sev.cpu())
      sev_precision.update(pred_sev.cpu(), lab_sev.cpu())
      sev_f1.update(pred_sev.cpu(), lab_sev.cpu())
      sev_recall.update(pred_sev.cpu(), lab_sev.cpu())

      sent_accuracy.update(pred_sent.cpu(), lab_sent.cpu())
      sent_precision.update(pred_sent.cpu(), lab_sent.cpu())
      sent_f1.update(pred_sent.cpu(), lab_sent.cpu())
      sent_recall.update(pred_sent.cpu(), lab_sent.cpu())
    comp_test_accuracy = comp_accuracy.compute()
    comp_test_precision = comp_precision.compute()
    comp_test_f1 = comp_f1.compute()
    comp_test_recall = comp_recall.compute()

    sev_test_accuracy = sev_accuracy.compute()
    sev_test_precision = sev_precision.compute()
    sev_test_f1 = sev_f1.compute()
    sev_test_recall = sev_recall.compute()

    emo_test_accuracy = emo_accuracy.compute()
    emo_test_precision = emo_precision.compute()
    emo_test_f1 = emo_f1.compute()
    emo_test_recall = emo_recall.compute()

    sent_test_accuracy = sent_accuracy.compute()
    sent_test_precision = sent_precision.compute()
    sent_test_f1 = sent_f1.compute()
    sent_test_recall = sent_recall.compute()
    
    print(f"comp_test_accuracy : {comp_test_accuracy}")
    print(f"comp_test_precision : {comp_test_precision}")
    print(f"comp_test_f1 : {comp_test_f1}")
    print(f"comp_test_recall : {comp_test_recall}")

    print(f"sent_test_accuracy : {sent_test_accuracy}")
    print(f"sent_test_precision : {sent_test_precision}")
    print(f"sent_test_f1 : {sent_test_f1}")
    print(f"sent_test_recall : {sent_test_recall}")

    print(f"emo_test_accuracy : {emo_test_accuracy}")
    print(f"emo_test_precision : {emo_test_precision}")
    print(f"emo_test_f1 : {emo_test_f1}")
    print(f"emo_test_recall : {emo_test_recall}")

    print(f"sev_test_accuracy : {sev_test_accuracy}")
    print(f"sev_test_precision : {sev_test_precision}")
    print(f"sev_test_f1 : {sev_test_f1}")
    print(f"sev_test_recall : {sev_test_recall}")

<ipython-input-29-d649379ce403>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids" : torch.tensor(self.input_ids[idx]).long(),
<ipython-input-29-d649379ce403>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask" : torch.tensor(self.attention_mask[idx]).long(),


RuntimeError: ignored